# pH-Sensitive Binder Design Evaluation

## Overview
This task evaluates your understanding and implementation of computational workflows for designing pH-sensitive protein binders, based on the strategies described in Ahn et al. 2025 "Computational Design of pH-Sensitive Binders".

## Two Design Strategies

### Strategy A: Interface Destabilization
- **Goal**: Weaken binder-target interaction at low pH
- **Mechanism**: Place histidine residues adjacent to positively charged residues (Arg/Lys) at the binding interface
- **Effect**: At low pH (~5.5), His becomes protonated (+1 charge), causing electrostatic repulsion with nearby Arg/Lys

### Strategy B: Monomer Destabilization  
- **Goal**: Destabilize the binder protein fold at low pH
- **Mechanism**: Install buried His-containing hydrogen bond networks (His-His, His-Arg, His-Lys) in the protein core
- **Effect**: At low pH, His protonation disrupts these buried H-bond networks, destabilizing the protein

## Task Structure
- **Part 0**: Conceptual understanding (multiple choice)
- **Part 1**: ProteinMPNN with histidine bias
- **Part 2**: Interface destabilization scoring & filtering (Strategy A)
- **Part 3**: Monomer destabilization with HBNet (Strategy B)


# Part 0: Conceptual Understanding

Answer the following multiple choice questions about pH-sensitive binder design.


## Question 0.1
**Why is histidine (His) the key residue for designing pH-sensitive proteins?**

A) His has the largest side chain among amino acids  
B) His has a pKa (~6.0) near physiological/endosomal pH, allowing protonation state changes  
C) His forms the strongest hydrogen bonds  
D) His is the most hydrophobic amino acid  

**Your answer:**


## Question 0.2
**In Strategy A (Interface Destabilization), which residue pairs create the pH-sensitive motif?**

A) His adjacent to Asp/Glu (negatively charged residues)  
B) His adjacent to Arg/Lys (positively charged residues)  
C) His adjacent to Ser/Thr (polar residues)  
D) His adjacent to Ala/Val (hydrophobic residues)  

**Your answer:**


## Question 0.3
**In Strategy B (Monomer Destabilization), where should the His-containing hydrogen bond networks be placed?**

A) At the protein surface, exposed to solvent  
B) At the binding interface with the target  
C) Buried in the protein core  
D) At the N-terminus of the protein  

**Your answer:**


## Question 0.4
**What network types are valid for Strategy B (Monomer Destabilization)?**

A) His-His, His-Arg, His-Lys networks  
B) Cys-Cys disulfide bonds  
C) Salt bridges between Asp and Lys  
D) Hydrophobic packing of Leu/Ile/Val  

**Your answer:**


## Question 0.5
**What is the filtering criterion for His H-bond energy in Strategy B?**

A) His H-bond energy > 0 kcal/mol (unfavorable)  
B) His H-bond energy < -0.5 kcal/mol (favorable)  
C) His H-bond energy = 0 kcal/mol (neutral)  
D) His H-bond energy > -5.0 kcal/mol  

**Your answer:**


---
# Part 1: ProteinMPNN with Histidine Bias

## Objective
Design sequences for a binder backbone using ProteinMPNN with histidine bias at specified interface positions.

## Background
ProteinMPNN is a deep learning method for protein sequence design. It can accept amino acid biases to favor certain residues at specific positions. For pH-sensitive binder design, we want to bias histidine at interface positions to enable Strategy A (Interface Destabilization).

## Task
1. Load the example binder backbone and target protein (IL-6)
2. Identify interface residues (within 8Å of target)
3. Run ProteinMPNN with histidine bias at interface positions
4. Analyze the designed sequences for His enrichment


In [ ]:
# Setup and imports
import os
import json
import numpy as np
from collections import Counter

# Check if ProteinMPNN is available
try:
    # ProteinMPNN can be called via subprocess or API
    PROTEINMPNN_AVAILABLE = True
    print("ProteinMPNN: Ready to use")
except:
    PROTEINMPNN_AVAILABLE = False
    print("ProteinMPNN: Not available - install from https://github.com/dauparas/ProteinMPNN")

# Data paths
DATA_DIR = "data"
TARGET_IL6 = os.path.join(DATA_DIR, "target_il6.pdb")  # PDB 1ALU

print(f"Target structure: {TARGET_IL6}")
print(f"File exists: {os.path.exists(TARGET_IL6)}")


## Task 1.1: Understanding ProteinMPNN Histidine Bias

ProteinMPNN supports amino acid biases through the `--bias_AA` flag or position-specific biases via `--bias_AA_per_residue`.

**Question**: Write the command-line argument to bias histidine with a log-odds score of +2.0 globally.

```bash
# Fill in the blank:
python ProteinMPNN/protein_mpnn_run.py \
    --pdb_path binder_complex.pdb \
    --out_folder output/ \
    ____________________________  # Add His bias argument here
```


In [ ]:
# Task 1.1: Your answer - write the bias argument
# Hint: The format is --bias_AA "X:value" where X is the amino acid letter

BIAS_ARGUMENT = ""  # TODO: Fill in the correct argument

print(f"Your bias argument: {BIAS_ARGUMENT}")


## Task 1.2: Position-Specific Histidine Bias

For more precise control, you can specify per-residue biases using a JSON file.

**Question**: Create a JSON dictionary that biases positions 10, 15, and 22 toward histidine (H) with a bias of +3.0, while keeping all other amino acids at 0.0 bias.


In [ ]:
# Task 1.2: Create position-specific bias dictionary
# The format should be: {chain_residue: {amino_acid: bias_value, ...}, ...}
# Example: {"A10": {"H": 3.0, "A": 0.0, ...}, "A15": {...}, "A22": {...}}

# Amino acid alphabet
AA_ALPHABET = "ACDEFGHIKLMNPQRSTVWY"

def create_his_bias_dict(positions, chain="A", his_bias=3.0):
    """
    Create a per-residue bias dictionary for ProteinMPNN.
    
    Args:
        positions: List of residue numbers to bias toward His
        chain: Chain identifier
        his_bias: Log-odds bias for histidine
    
    Returns:
        Dictionary in ProteinMPNN bias format
    """
    bias_dict = {}
    
    # TODO: Implement this function
    # For each position, create a dictionary with His biased and others at 0
    
    return bias_dict

# Test with positions 10, 15, 22
interface_positions = [10, 15, 22]
bias_dict = create_his_bias_dict(interface_positions, his_bias=3.0)

print("Position-specific bias dictionary:")
print(json.dumps(bias_dict, indent=2))


---
# Part 2: Interface Destabilization Scoring & Filtering (Strategy A)

## Objective
Implement a comprehensive scoring and filtering pipeline to identify promising Interface Destabilization candidates.

## Filtering Criteria (from paper workflow)
1. **His-cation motif**: At least one His accepting H-bond from Arg/Lys at interface
2. **Rosetta ddG_elec >= 0**: Electrostatic energy should be unfavorable at low pH (protonated His)
3. **Additional metrics**: dSASA_int, packstat, total_score after FastRelax

## Data
- Target: PCSK9 (PDB 2P4E)


In [ ]:
# Part 2: Setup PyRosetta
import warnings
warnings.filterwarnings('ignore')

# Initialize PyRosetta (uncomment when running)
# import pyrosetta
# pyrosetta.init("-mute all")

# For demonstration, we'll use BioPython for structure analysis
from Bio.PDB import PDBParser, NeighborSearch, Selection
import os

TARGET_PCSK9 = os.path.join(DATA_DIR, "target_pcsk9.pdb")
print(f"Target PCSK9: {TARGET_PCSK9}")
print(f"File exists: {os.path.exists(TARGET_PCSK9)}")


## Task 2.1: Detect His-Cation Interface Motif

Implement a function to detect histidine residues that are:
1. At the binding interface (within 5Å of target)
2. Within H-bond distance (< 3.5Å) of a positively charged residue (Arg or Lys)


In [ ]:
def detect_his_cation_motif(structure, binder_chain="A", target_chain="B", 
                            interface_cutoff=5.0, hbond_cutoff=3.5):
    """
    Detect His-Arg/Lys motifs at the binding interface.
    
    Args:
        structure: BioPython Structure object
        binder_chain: Chain ID of the binder
        target_chain: Chain ID of the target
        interface_cutoff: Distance cutoff for interface residues (Å)
        hbond_cutoff: Distance cutoff for H-bond detection (Å)
    
    Returns:
        List of dictionaries containing His-cation pairs found
    """
    his_cation_pairs = []
    
    # TODO: Implement this function
    # Steps:
    # 1. Get all atoms from binder and target chains
    # 2. Find interface residues (binder residues within interface_cutoff of target)
    # 3. For each His at interface, find nearby Arg/Lys within hbond_cutoff
    # 4. Check if the His nitrogen atoms (ND1, NE2) are within H-bond distance
    
    return his_cation_pairs

# Test the function (placeholder - needs actual binder-target complex)
print("Function defined. Needs binder-target complex PDB to test.")


## Task 2.2: PyRosetta FastRelax and Interface Analysis

Implement the Rosetta-based scoring pipeline:
1. FastRelax the binder-target complex
2. Calculate interface energy metrics using InterfaceAnalyzer
3. Extract fa_elec (electrostatic) energy decomposition


In [ ]:
def score_interface_destabilization(pdb_path, binder_chain="A", target_chain="B"):
    """
    Score a binder-target complex for interface destabilization potential.
    
    Uses PyRosetta to:
    1. FastRelax the complex
    2. Calculate interface metrics
    3. Decompose electrostatic energies
    
    Args:
        pdb_path: Path to binder-target complex PDB
        binder_chain: Chain ID of binder
        target_chain: Chain ID of target
    
    Returns:
        Dictionary of scores: {
            'total_score': float,
            'dG_separated': float,
            'dSASA_int': float,
            'fa_elec': float,
            'packstat': float
        }
    """
    scores = {}
    
    # TODO: Implement using PyRosetta
    # Pseudocode:
    """
    import pyrosetta
    from pyrosetta.rosetta.protocols.relax import FastRelax
    from pyrosetta.rosetta.protocols.analysis import InterfaceAnalyzerMover
    
    # Load pose
    pose = pyrosetta.pose_from_pdb(pdb_path)
    
    # Setup score function
    sfxn = pyrosetta.get_fa_scorefxn()
    
    # FastRelax
    relax = FastRelax()
    relax.set_scorefxn(sfxn)
    relax.apply(pose)
    
    # Interface analysis
    interface = InterfaceAnalyzerMover(binder_chain + "_" + target_chain)
    interface.apply(pose)
    
    # Extract scores
    scores['total_score'] = sfxn(pose)
    scores['dG_separated'] = pose.scores['dG_separated']
    scores['dSASA_int'] = pose.scores['dSASA_int']
    
    # Get fa_elec for interface residues
    # ... energy decomposition code ...
    """
    
    return scores

print("PyRosetta scoring function defined.")


In [ ]:
def filter_interface_destabilization_candidates(pdb_files, 
                                                 min_his_cation_pairs=1,
                                                 min_ddg_elec=0.0):
    """
    Filter binder candidates for interface destabilization strategy.
    
    Criteria:
    1. At least min_his_cation_pairs His-Arg/Lys pairs at interface
    2. ddG_elec >= min_ddg_elec (electrostatic penalty at low pH)
    
    Args:
        pdb_files: List of PDB file paths to evaluate
        min_his_cation_pairs: Minimum number of His-cation pairs required
        min_ddg_elec: Minimum electrostatic ddG threshold
    
    Returns:
        List of passing candidates with their scores
    """
    passing_candidates = []
    
    for pdb_path in pdb_files:
        # TODO: Implement filtering logic
        # 1. Load structure
        # 2. Check His-cation motif criterion
        # 3. Score with PyRosetta
        # 4. Check ddG_elec criterion
        # 5. If passes all, add to passing_candidates
        pass
    
    return passing_candidates

# Example usage (needs actual candidate PDBs)
print("Filtering pipeline defined.")
print("\\nFiltering criteria:")
print("  1. >= 1 His-Arg/Lys pair at interface")
print("  2. ddG_elec >= 0 (electrostatic penalty at low pH)")


---
# Part 3: Monomer Destabilization with HBNet (Strategy B)

## Objective
Use PyRosetta HBNet to design buried His-containing hydrogen bond networks in a protein scaffold.

## Background
HBNet (Hydrogen Bond Network) is a Rosetta protocol for designing networks of hydrogen-bonded residues. For Strategy B, we want to install buried networks containing:
- His-His pairs
- His-Arg pairs  
- His-Lys pairs

These networks are stable at neutral pH but become destabilized when His becomes protonated at low pH.

## Data
- Scaffold: De novo 3-helix bundle (PDB 5L33)


In [ ]:
# Part 3: Setup
SCAFFOLD_PDB = os.path.join(DATA_DIR, "scaffold.pdb")  # PDB 5L33
print(f"Scaffold: {SCAFFOLD_PDB}")
print(f"File exists: {os.path.exists(SCAFFOLD_PDB)}")


## Task 3.1: Identify Core Residues for HBNet

First, identify buried core residues suitable for installing His-containing networks.
A residue is considered "buried" if its relative SASA < 25%.


In [ ]:
def identify_core_residues(pdb_path, sasa_threshold=0.25):
    """
    Identify buried core residues suitable for HBNet installation.
    
    Args:
        pdb_path: Path to PDB file
        sasa_threshold: Relative SASA threshold (residues below this are "buried")
    
    Returns:
        List of (chain, resnum, resname) tuples for core residues
    """
    core_residues = []
    
    # TODO: Implement using FreeSASA or BioPython SASA
    # Steps:
    # 1. Load structure
    # 2. Calculate per-residue SASA
    # 3. Calculate relative SASA (SASA / max_SASA for that residue type)
    # 4. Filter for residues with relative SASA < threshold
    
    # Pseudocode:
    """
    import freesasa
    
    # Standard max SASA values (Å²) for amino acids
    MAX_SASA = {
        'ALA': 129, 'ARG': 274, 'ASN': 195, 'ASP': 193, 'CYS': 167,
        'GLN': 225, 'GLU': 223, 'GLY': 104, 'HIS': 224, 'ILE': 197,
        'LEU': 201, 'LYS': 236, 'MET': 224, 'PHE': 240, 'PRO': 159,
        'SER': 155, 'THR': 172, 'TRP': 285, 'TYR': 263, 'VAL': 174
    }
    
    structure = freesasa.Structure(pdb_path)
    result = freesasa.calc(structure)
    
    for residue in structure.residues():
        sasa = result.residueAreas()[residue]
        max_sasa = MAX_SASA.get(residue.resname, 200)
        rel_sasa = sasa / max_sasa
        
        if rel_sasa < sasa_threshold:
            core_residues.append((residue.chain, residue.resnum, residue.resname))
    """
    
    return core_residues

# Test (placeholder)
print("Core residue identification function defined.")


## Task 3.2: Configure and Run HBNet

Configure PyRosetta HBNet to find His-containing hydrogen bond networks in the protein core.


In [ ]:
def setup_hbnet_for_his_networks(pose, core_positions, network_types=["HIS_HIS", "HIS_ARG", "HIS_LYS"]):
    """
    Configure HBNet to design His-containing hydrogen bond networks.
    
    Args:
        pose: PyRosetta Pose object
        core_positions: List of residue positions to target for network installation
        network_types: Types of networks to design (His-His, His-Arg, His-Lys)
    
    Returns:
        Configured HBNet mover
    """
    # TODO: Implement HBNet configuration
    # This requires PyRosetta with HBNet protocol
    
    # Pseudocode:
    """
    from pyrosetta.rosetta.protocols.hbnet import HBNet
    from pyrosetta.rosetta.core.select.residue_selector import ResidueIndexSelector
    
    # Create residue selector for core positions
    core_selector = ResidueIndexSelector(",".join(map(str, core_positions)))
    
    # Configure HBNet
    hbnet = HBNet()
    hbnet.set_core_selector(core_selector)
    
    # Set allowed residue types for network
    # For His-containing networks, allow HIS, ARG, LYS
    allowed_aas = "HRK"  # His, Arg, Lys (one-letter codes)
    hbnet.set_allowed_aas(allowed_aas)
    
    # Require at least one His in each network
    hbnet.set_min_network_size(2)
    hbnet.set_min_his_in_network(1)
    
    return hbnet
    """
    
    return None  # Placeholder

print("HBNet configuration function defined.")


In [ ]:
def score_hbnet_designs(pose, networks, min_his_hbond_energy=-0.5):
    """
    Score HBNet designs and filter based on His H-bond energy.
    
    Args:
        pose: PyRosetta Pose with HBNet-designed networks
        networks: List of network definitions from HBNet
        min_his_hbond_energy: Minimum H-bond energy threshold (kcal/mol)
    
    Returns:
        List of passing networks with scores
    """
    passing_networks = []
    
    # TODO: Implement scoring
    # Pseudocode:
    """
    from pyrosetta.rosetta.core.scoring.hbonds import HBondSet
    
    # Get H-bond set from pose
    hbond_set = HBondSet(pose)
    
    for network in networks:
        # Find His residues in network
        his_residues = [r for r in network.residues if pose.residue(r).name3() == "HIS"]
        
        for his_res in his_residues:
            # Get H-bonds involving this His
            his_hbonds = [hb for hb in hbond_set if his_res in (hb.don_res, hb.acc_res)]
            
            # Sum H-bond energies
            total_his_hbond_energy = sum(hb.energy for hb in his_hbonds)
            
            if total_his_hbond_energy < min_his_hbond_energy:
                passing_networks.append({
                    'network': network,
                    'his_residue': his_res,
                    'hbond_energy': total_his_hbond_energy,
                    'network_size': len(network.residues)
                })
    """
    
    return passing_networks

print("HBNet scoring function defined.")
print(f"\\nFiltering criterion: His H-bond energy < -0.5 kcal/mol")


## Task 3.4: Complete Monomer Destabilization Pipeline

Combine all steps into a complete pipeline.


In [ ]:
def run_monomer_destabilization_pipeline(pdb_path, output_dir="hbnet_output"):
    """
    Complete pipeline for monomer destabilization design using HBNet.
    
    Workflow:
    1. Load scaffold structure
    2. Identify core residues (SASA < 25%)
    3. Run HBNet to design His-containing networks
    4. Score and filter networks (His H-bond energy < -0.5 kcal/mol)
    5. Output passing designs
    
    Args:
        pdb_path: Path to scaffold PDB
        output_dir: Directory for output files
    
    Returns:
        List of passing HBNet designs
    """
    print(f"Running monomer destabilization pipeline...")
    print(f"Input: {pdb_path}")
    print(f"Output: {output_dir}")
    
    # TODO: Implement complete pipeline
    # 1. core_residues = identify_core_residues(pdb_path)
    # 2. pose = pyrosetta.pose_from_pdb(pdb_path)
    # 3. hbnet = setup_hbnet_for_his_networks(pose, core_residues)
    # 4. hbnet.apply(pose)
    # 5. networks = hbnet.get_networks()
    # 6. passing = score_hbnet_designs(pose, networks)
    # 7. Save passing designs
    
    passing_designs = []
    return passing_designs

# Example usage
print("Complete pipeline function defined.")
print("\\nPipeline steps:")
print("  1. Identify core residues (SASA < 25%)")
print("  2. Run HBNet for His-His, His-Arg, His-Lys networks")
print("  3. Filter: His H-bond energy < -0.5 kcal/mol")
print("  4. Output passing designs")


---
# Summary

This task covered the key computational workflows for pH-sensitive binder design:

## Part 0: Conceptual Understanding
- Histidine's role (pKa ~6.0)
- His-cation motifs for interface destabilization
- Buried His networks for monomer destabilization

## Part 1: ProteinMPNN with Histidine Bias
- Global bias with `--bias_AA "H:2.0"`
- Position-specific bias with JSON dictionaries

## Part 2: Interface Destabilization (Strategy A)
- His-Arg/Lys motif detection at interface
- PyRosetta FastRelax and InterfaceAnalyzer
- Filter: His-cation pair + ddG_elec >= 0

## Part 3: Monomer Destabilization (Strategy B)
- Core residue identification (SASA < 25%)
- HBNet for His-His, His-Arg, His-Lys networks
- Filter: His H-bond energy < -0.5 kcal/mol
